In [ ]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 5.0 MB/s 


In [ ]:
# Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd # Pandas to create small dataframes 
import numpy as np
import datetime # Convert to unix time
import time # Convert to unix time
# If numpy is not installed already : pip3 install numpy
import numpy as np # Do aritmetic operations on arrays
# Matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns # Plots
from matplotlib import rcParams # Size of plots  
import math
import pickle
import os
# To install xgboost: pip3 install xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "gdrive/My Drive/Major Project/wiki/"

In [ ]:
df_train = pd.read_csv(data_path+"pos_train_91.csv", index_col=0)

In [ ]:
df_train.head()

,src,dest,link
33079,1332,1211,1
100635,6224,3649,1
61866,2909,2439,1
86957,4753,2163,1
103670,7082,5806,1


In [ ]:
import tensorflow

In [ ]:
from tensorflow import keras

In [ ]:
from stellargraph.core import StellarGraph
from stellargraph.core import StellarDiGraph
from stellargraph.mapper import AdjacencyPowerGenerator
from stellargraph.layer import WatchYourStep
from stellargraph.losses import graph_log_likelihood
from stellargraph.utils import plot_history

from tensorflow.keras import optimizers, Model, layers, regularizers
import tensorflow as tf

In [ ]:
from stellargraph.data import EdgeSplitter, UniformRandomWalk, UnsupervisedSampler
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())

In [ ]:
degrees = nx.degree(train_graph)

In [ ]:
degrees = dict(degrees)

In [ ]:
import collections
features = {}
for key, val in degrees.items():
  features[key] = [val]

In [ ]:
pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

In [ ]:
katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

In [ ]:
nx.set_node_attributes(train_graph, features, "features")

In [ ]:
G = StellarGraph.from_networkx(train_graph, node_features="features")

In [ ]:
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 7115, Edges: 93422

 Node types:
  default: [7115]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [93422]
        Weights: all 1 (default)
        Features: none


In [ ]:
nodes = list(G.nodes())
num_walks = 1
length = 5

In [ ]:
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length= length, number_of_walks = num_walks
)

In [ ]:
batch_size=128
epochs=50
num_samples= [10,5]

In [ ]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

In [ ]:
layer_sizes = [64, 64]
graphsage = GraphSAGE(
    layer_sizes = layer_sizes, generator = generator, bias = True, normalize='l2'
)

In [ ]:
x_inp, x_out = graphsage.in_out_tensors()

In [ ]:
prediction = link_classification(
    output_dim = 1, output_act = "sigmoid", edge_embedding_method = "ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [ ]:
model = keras.Model(inputs = x_inp, outputs = prediction)
model.compile(
    optimizer = keras.optimizers.Adam(1e-2),
    loss = keras.losses.binary_crossentropy,
    metrics = [keras.metrics.binary_accuracy]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor='loss',
    verbose=1,
    restore_best_weights=True,
    patience=3
)

In [ ]:
history = model.fit(
    train_gen,
    epochs=epochs,
    callbacks=[es],
    verbose=1,
    use_multiprocessing=True,
    shuffle=True,
)
filep = open(data_path+"graphsage_model.h5", "wb")
pickle.dump(model, filep)
filep.close()

Epoch 1/50
445/445 [==============================] - 47s 92ms/step - loss: 0.7321 - binary_accuracy: 0.5002
Epoch 2/50
445/445 [==============================] - 37s 83ms/step - loss: 0.7290 - binary_accuracy: 0.5019
Epoch 3/50
445/445 [==============================] - 43s 95ms/step - loss: 0.7270 - binary_accuracy: 0.5031
Epoch 4/50
445/445 [==============================] - 38s 85ms/step - loss: 0.7270 - binary_accuracy: 0.5071
Epoch 5/50
445/445 [==============================] - 37s 82ms/step - loss: 0.7160 - binary_accuracy: 0.5205
Epoch 6/50
445/445 [==============================] - 36s 80ms/step - loss: 0.7120 - binary_accuracy: 0.5233
Epoch 7/50
445/445 [==============================] - 35s 78ms/step - loss: 0.7098 - binary_accuracy: 0.5230
Epoch 8/50
445/445 [==============================] - 36s 80ms/step - loss: 0.7077 - binary_accuracy: 0.5288
Epoch 9/50
445/445 [==============================] - 38s 86ms/step - loss: 0.7089 - binary_accuracy: 0.5270
Epoch 10/50
445/445

In [ ]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [ ]:
from stellargraph.mapper.sampled_node_generators import GraphSAGENodeGenerator
node_ids = list(sorted(G.nodes()))
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

In [ ]:
node_emb = embedding_model.predict(node_gen, verbose=1)

56/56 [==============================] - 2s 40ms/step


In [ ]:
type(node_emb)

numpy.ndarray

In [ ]:
node_emb.shape

(7115, 64)

In [ ]:
node_emb[0].reshape(1,64).shape

(1, 64)

In [ ]:
emb_dict = {}
for i, node in enumerate(node_ids):
  emb_dict[node] = np.array(node_emb[i].reshape(1,64))

In [ ]:
combined = pd.read_csv(data_path+"/train_91.csv", index_col = 0)
combined.head()

,src,dest,link
11538,308,75,1
2338,3874,251,0
89956,1506,4335,0
54348,2471,4066,1
91913,1584,2951,0


In [ ]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)

0it [00:00, ?it/s]

In [ ]:
train_y = combined['link']
train_y.shape

(187022,)

In [ ]:
combined_test = pd.read_csv(data_path+"/test_91.csv", index_col = 0)
combined_test.head()

,src,dest,link
13017,415,81,1
98801,2230,1590,0
4421,67,4680,1
57102,2633,2303,1
62449,2999,698,1


In [ ]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_test.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])

print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_test['link'] 

0it [00:00, ?it/s]

<class 'list'>
(20667, 64)


In [ ]:
train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [ ]:
train_emb_sample.shape

(93511, 64)

In [ ]:
#reading
from pandas import read_hdf
df_final_train = read_hdf(data_path + 'fea_sample/storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf(data_path + 'fea_sample/storage_sample_stage4.h5', 'test_df',mode='r')

In [ ]:
df_final_train.columns

Index(['src', 'dest', 'link', 'jaccard_followers', 'jaccard_followees',
       'cosine_followers', 'cosine_followees', 'num_followers_s',
       'num_followers_d', 'num_followees_s', 'num_followees_d',
       'inter_followers', 'inter_followees', 'adar_index', 'follows_back',
       'same_comp', 'shortest_path', 'weight_in', 'weight_out', 'weight_f1',
       'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s', 'page_rank_d',
       'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4',
       'svd_u_s_5', 'svd_u_s_6', 'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3',
       'svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6', 'svd_v_s_1', 'svd_v_s_2',
       'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6', 'svd_v_d_1',
       'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5', 'svd_v_d_6',
       'svd_dot_u', 'svd_dot_v', 'pref_attach_er', 'pref_attach_ee'],
      dtype='object')

In [ ]:
df_final_train.shape

(187022, 59)

In [ ]:
y_train = df_final_train.link
y_test = df_final_test.link

In [ ]:
df_final_train.drop(['src', 'dest','link'],axis=1,inplace=True)
df_final_test.drop(['src', 'dest','link'],axis=1,inplace=True)

In [ ]:
# GraphSAGE only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9122936127041531 test Score 0.8987366003062788


In [ ]:
# GraphSAGE only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.922935006061884 test Score 0.8993198582239679


In [ ]:
# GraphSAGE only
from lightgbm import LGBMClassifier
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.90636455917236 test Score 0.8991106435880272


In [ ]:
# Heuristic only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9856018983688566 test Score 0.9665368882755901


In [ ]:
# Heuristic only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.981759433534703 test Score 0.9604033086975674


In [ ]:
# Heuristic only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9844932244688583 test Score 0.9676879125209339


In [ ]:
df_final_train = np.hstack((df_final_train, train_emb))
print(df_final_train.shape)
df_final_test = np.hstack((df_final_test, test_emb))
print(df_final_test.shape)

(187022, 120)
(20667, 120)


In [ ]:
print(type(train_y), type(y_train))
for i in range(len(y_train.values)):
  if train_y.values[i] != y_train.values[i]: print(i)

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [ ]:
# Heuristic + GraphSAGE
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9858432608579517 test Score 0.9661150512214342


In [ ]:
# Heuristic + GraphSAGE
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9801498047869631 test Score 0.9560386002534361


In [ ]:
# Heuristic + GraphSAGE
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9853134519801187 test Score 0.9671921182266009
